## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [104]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats
import sklearn as sk
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


## Test Hypothesis

In [51]:
purch_treat = train_data[(train_data['Promotion']=='Yes') & (train_data['purchase']==1)].shape[0]
cust_treat = train_data[train_data['Promotion']=='Yes'].shape[0]
print(purch_treat, cust_treat, purch_treat/cust_treat)

purch_cont = train_data[(train_data['Promotion']=='No') & (train_data['purchase']==1)].shape[0]
cust_cont = train_data[train_data['Promotion']=='No'].shape[0]
print(purch_cont, cust_cont, purch_cont/cust_cont)

irr = (purch_treat / cust_treat) - (purch_cont / cust_cont)
irr

721 42364 0.017019167217448776
319 42170 0.007564619397676073


0.009454547819772702

In [52]:
alpha = 0.05
p_total = (purch_cont + purch_treat) / (cust_cont + cust_treat)
se = np.sqrt(p_total * (1 - p_total) * (1 / cust_cont + 1 / cust_treat))
z_pool = irr/se
p_val_ctr = 1 - stats.norm.cdf(z_pool)
print('statistically significant' if p_val_ctr < alpha else 'statistically not significant')

statistically significant


In [53]:
0.15*cust_treat

6354.599999999999

In [54]:
nir = (10*purch_treat - 0.15*cust_treat) - 10*purch_cont
nir

-2334.5999999999995

In [55]:
alphaBC = alpha / 2
se = np.sqrt(p_total * (1 - p_total) * (1 / cust_cont + 1 / cust_treat))
z_pool = nir/se
p_val_ctr = 1 - stats.norm.cdf(z_pool)
print('statistically significant' if p_val_ctr < alphaBC else 'statistically not significant')

statistically not significant


In [56]:
train_data['V1'].value_counts()

1    31631
2    31608
3    10670
0    10625
Name: V1, dtype: int64

In [57]:
print(train_data['V2'].min())
print(train_data['V2'].max())

7.10400719
50.3759126


In [58]:
print(train_data['V3'].min())
print(train_data['V3'].max())

-1.6845501
1.6919836


In [59]:
train_data['V4'].value_counts()

2    57450
1    27084
Name: V4, dtype: int64

In [60]:
train_data['V5'].value_counts()

3    32743
2    31196
1    15412
4     5183
Name: V5, dtype: int64

In [61]:
train_data['V6'].value_counts()

3    21186
4    21176
2    21146
1    21026
Name: V6, dtype: int64

In [62]:
train_data['V7'].value_counts()

2    59317
1    25217
Name: V7, dtype: int64

In [63]:
train_data.loc[train_data['Promotion']=='No', 'Promotion'] = 0
train_data.loc[train_data['Promotion']=='Yes', 'Promotion'] = 1
train_data['Promotion'].value_counts()

1    42364
0    42170
Name: Promotion, dtype: int64

In [64]:
feature_cols = train_data.iloc[:,3:].columns.tolist()
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,0,0,2,30.443518,-1.165083,1,1,3,2
1,3,0,0,3,32.159350,-0.645617,2,3,2,2
2,4,0,0,2,30.431659,0.133583,1,1,4,2
3,5,0,0,0,26.588914,-0.212728,2,1,4,2
4,8,1,0,3,28.044332,-0.385883,1,1,2,2


In [65]:
# load data
X = train_data[feature_cols]
y = train_data['purchase']

# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [66]:
# pipeline = Pipeline([('scaler', MinMaxScaler()), 
#                  ('clf', RandomForestClassifier(n_estimators=100))])

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [105]:
# clf=RandomForestClassifier(n_estimators=10, criterion='gini')
# clf=DecisionTreeClassifier(random_state=0)
# ExtraTreesClassifier(n_estimators=100, max_depth=None, random_state=0)
BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [106]:
print(metrics.accuracy_score(y_test, y_pred))

0.972577696527


In [98]:
# clf.get_params()

In [99]:
# parameters = dict(bootstrap=['True','False'], criterion=['gini','entropy'])

# cv = GridSearchCV(clf, param_grid=parameters)

In [107]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    df = scaler.fit_transform(df)
    print(df)
    promotion= clf.predict(df)
    promotion = np.where(promotion == 0 , 'No', promotion)
    promotion = np.where(promotion == '1' , 'Yes', promotion)
    print(np.unique(promotion, return_counts=True))
    return promotion

In [108]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

[[ 0.33333333  0.70087943  0.84615384 ...,  0.          0.33333333  1.        ]
 [ 0.33333333  0.34769774  0.69230769 ...,  0.33333333  0.33333333  1.        ]
 [ 0.33333333  0.37798162  0.02564103 ...,  0.66666667  1.          1.        ]
 ..., 
 [ 1.          0.40275983  0.84615384 ...,  0.33333333  0.          1.        ]
 [ 0.66666667  0.5032981   0.66666666 ...,  0.33333333  1.          0.        ]
 [ 0.33333333  0.46481814  1.         ...,  0.66666667  0.          1.        ]]
(array(['No', 'Yes'], 
      dtype='<U21'), array([41016,   634]))
Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0185.

Your nir with this strategy is 11.40.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.018518518518518517, 11.399999999999999)